In [2]:
import pandas as pd
import random as rd
import numpy as np
import re

In [5]:
def separarTreinoTeste(dfA):
    dfB = pd.DataFrame()
    dfC = pd.DataFrame()
    for i in range (len(dfA)):
        vetor = dfA.iloc[i, 0]
        if (rd.random() > 0.8): # vai p teste
            dfB = pd.concat([dfB, pd.DataFrame(data=[[vetor]], columns=['noticias'])], ignore_index=True)
        else:
            dfC = pd.concat([dfC, pd.DataFrame(data=[[vetor]], columns=['noticias'])], ignore_index=True)

    return dfB, dfC

In [6]:
def prepararDfTeste (dfA, dfB):
    dados = []

    # embaralha os dfs
    dfEmbaralhado = pd.concat([dfA, dfB], ignore_index=True)
    indices = list(dfEmbaralhado.index)
    rd.shuffle(indices)
    dfEmbaralhado = dfEmbaralhado.iloc[indices].reset_index(drop=True)
    
    # cria uma coluna pra tag
    for i in range (len(dfEmbaralhado)):
        news = dfEmbaralhado.iloc[i, 0]
        tag = news[0]
        news = news[1:]
        dados.append([tag, news])

    return pd.DataFrame(dados, columns=['tag', 'noticia'])

In [7]:
def prepararDfsTreino (dfF, dfR):
    for i in range(len(dfF)):
        noticia = dfF.iloc[i, 0]
        noticia.remove('f')

    for i in range(len(dfR)):
        noticia = dfR.iloc[i, 0]
        noticia.remove('r')

    palavras = []
    for i in range(len(dfF)):
        vetor = dfF.iloc[i, 0]
        for word in vetor:
            palavras.append(word)
    dfF = pd.DataFrame({'palavras': palavras})
    dfF = dfF.groupby('palavras').size().reset_index(name='aparicoesFalsas')

    palavras = []
    for i in range(len(dfR)):
        vetor = dfR.iloc[i, 0]
        for word in vetor:
            palavras.append(word)
    dfR = pd.DataFrame({'palavras': palavras})
    dfR = dfR.groupby('palavras').size().reset_index(name='aparicoesReais')

    return dfF, dfR

In [49]:
def evaluate(dfReal, dfFalso, noticiasTeste):
    # Junta os dataframes e une as palavras
    dfJunto = dfReal.merge(dfFalso, how='outer')

    # Conta quantas palavras tem em uma e não tem na outra
    nReal = dfJunto.loc[dfJunto['aparicoesReais'].isna(), 'palavras'].count()
    nFalsa = dfJunto.loc[dfJunto['aparicoesFalsas'].isna(), 'palavras'].count()

    # Preenche as palavras que não aparecem com 1
    dfJunto.fillna(1, inplace=True)

    # Soma as palavras com 0 aparições com 1 para evitar multiplicação por 0
    dfJunto.loc[dfJunto['aparicoesReais'] > 1, 'aparicoesReais'] += 1
    dfJunto.loc[dfJunto['aparicoesFalsas'] > 1, 'aparicoesFalsas'] += 1

    # Soma o total de aparições
    totalReal = dfJunto['aparicoesReais'].sum()
    totalFalso = dfJunto['aparicoesFalsas'].sum()

    # Balanceia a ordem de grandeza das aparições
    dfJunto['aparicoesFalsas'] *= totalReal / totalFalso

    # Faz o coeficiente de bray courtis
    dfJunto['bc'] = abs(dfJunto['aparicoesReais'] - dfJunto['aparicoesFalsas']) / (dfJunto['aparicoesReais'] + dfJunto['aparicoesFalsas'])

    # Limpa palavras com coeficiente menor que 0.3 (palavras com aparições equivalentes em ambos casos)
    dfJunto = dfJunto.loc[dfJunto['bc'] > 0.3]

    # Pega o total de aparições após NLP
    tFalsas = dfJunto['aparicoesFalsas'].sum()
    tReais = dfJunto['aparicoesReais'].sum()

    # Define um df de teste
    dfJuntoTeste = dfJunto

    # Faz os calculos das probabilidades
    dfJuntoTeste['aparicoesFalsas'] = dfJunto['aparicoesFalsas'] / (nFalsa + tFalsas)
    dfJuntoTeste['aparicoesReais'] = dfJunto['aparicoesReais'] / (nReal + tReais)

    # Pega o log na base 10
    dfJuntoTeste['aparicoesFalsas'] = np.log10(dfJuntoTeste['aparicoesFalsas'])
    dfJuntoTeste['aparicoesReais'] = np.log10(dfJuntoTeste['aparicoesReais'])

    # NOTICIA:
    
    indices = ['fF', 'fR', 'rF', 'rR']
    dfDados = pd.DataFrame(0, index=indices, columns=['num'])

    for i in range (len(noticiasTeste)):
        noticia = noticiasTeste.iloc[i, 1]
        tag = noticiasTeste.iloc[i, 0]

        # Cria um df com a notícia limpa
        dfNoticia = pd.DataFrame(data = noticia, columns=['palavra'])

        # Junta o df da noticia com o das probabilidades
        dfNoticia = dfNoticia.merge(dfJuntoTeste, how='inner', left_on='palavra', right_on='palavras')

        # Faz a soma das probabilidades
        iFalsa = dfNoticia['aparicoesFalsas'].sum()
        iReal = dfNoticia['aparicoesReais'].sum()

        # Confere o resultado
        if (iFalsa > iReal): # Falsa
            resultado = 'F'
        else:
            resultado = 'R'
            
        chave = tag + resultado
        dfDados.loc[chave, 'num'] += 1
    
    acuracia = round(float((dfDados.loc['fF', 'num'] + dfDados.loc['rR', 'num']) / dfDados['num'].sum()), 4)
    
    sensiReal = round(float(dfDados.loc['rR', 'num'] / (dfDados.loc['rR', 'num'] + dfDados.loc['rF', 'num'])), 4)
    sensiFalsa = round(float(dfDados.loc['fF', 'num'] / (dfDados.loc['fF', 'num'] + dfDados.loc['fR', 'num'])), 4)
    
    precReal = round(float(dfDados.loc['rR', 'num'] / (dfDados.loc['rR', 'num'] + dfDados.loc['fR', 'num'])), 4)
    precFalsa = round(float(dfDados.loc['fF', 'num'] / (dfDados.loc['fF', 'num'] + dfDados.loc['rF', 'num'])), 4)
    
    return acuracia, sensiReal, precReal, sensiFalsa, precFalsa

In [38]:
def calcularDPMedia(dados):
    n = len(dados)
    media = sum(dados) / n

    somaDiferencaQuadrados = 0

    for valor in dados:
        diferenca = valor - media
        somaDiferencaQuadrados += diferenca ** 2

    vAmostral = somaDiferencaQuadrados / (n-1)
    dp = vAmostral ** 0.5
    return dp, media

In [34]:
def calcularIntervaloConfianca(dp, media, n):
    min = media - (1.96 * dp) / n ** 0.5
    max = media + (1.96 * dp) / n ** 0.5
    return min, max

In [51]:
acuracias = []
sensisReais = []
sensisFalsas = []
precsReais = []
precsFalsas = []


for i in range(36):
    dfFalso = pd.read_json('../data/train/fakeTrain.json', orient='records', lines=True)
    dfReal = pd.read_json('../data/train/realTrain.json', orient='records', lines=True)

    [dfFalsoTeste, dfFalsoTreino] = separarTreinoTeste(dfFalso)
    [dfRealTeste, dfRealTreino] = separarTreinoTeste(dfReal)

    dfTestes = prepararDfTeste(dfFalsoTeste, dfRealTeste)
    [dfFalsoTreino, dfRealTreino] = prepararDfsTreino(dfFalsoTreino, dfRealTreino)

    [acuracia, sensiReal, precReal, sensiFalsa, precFalsa] = evaluate(dfRealTreino, dfFalsoTreino, dfTestes)

    acuracias.append(acuracia)
    sensisReais.append(sensiReal)
    precsReais.append(precReal)

    sensisFalsas.append(sensiFalsa)
    precsFalsas.append(precFalsa)

In [60]:
dpPrecFalsas, mediaPrecFalsas = calcularDPMedia(precsFalsas)
minPrecisaoFalsas, maxPrecisaoFalsas = calcularIntervaloConfianca(dpPrecFalsas, mediaPrecFalsas, len(precsFalsas))

dpPrecReais, mediaPrecFReais= calcularDPMedia(precsReais)
minPrecisaoReais, maxPrecisaoReais = calcularIntervaloConfianca(dpPrecReais, mediaPrecFReais, len(precsReais))

dpSensiReais, mediaSensiReais= calcularDPMedia(sensisReais)
minSensiReais, maxSensiReais = calcularIntervaloConfianca(dpSensiReais, mediaSensiReais, len(sensisReais))

dpSensiFalsas, mediaSensiFalsas= calcularDPMedia(sensisFalsas)
minSensiFalsas, maxSensiFalsas = calcularIntervaloConfianca(dpSensiFalsas, mediaSensiFalsas, len(sensisFalsas))

dpAcuracidade, mediaAcuracidade= calcularDPMedia(acuracias)
minAcuracidade, maxAcuracidade = calcularIntervaloConfianca(dpAcuracidade, mediaAcuracidade, len(acuracias))


In [64]:
def formatar_porcentagem(valor):
    return f"{valor * 100:.2f}%"

dados = {
    'Métrica': ['Precisão Falsas', 'Precisão Reais', 'Sensibilidade Reais', 'Sensibilidade Falsas', 'Acuracidade'],
    'Mínimo': list([
        minPrecisaoFalsas,
        minPrecisaoReais,
        minSensiReais,
        minSensiFalsas,
        minAcuracidade
    ]),
    'Máximo': list([
        maxPrecisaoFalsas,
        maxPrecisaoReais,
        maxSensiReais,
        maxSensiFalsas,
        maxAcuracidade
    ])
}

dfIntervalos = pd.DataFrame(dados)

dfIntervalos['Mínimo'] = dfIntervalos['Mínimo'].round(4)
dfIntervalos['Máximo'] = dfIntervalos['Máximo'].round(4)

dfIntervalos

,Métrica,Mínimo,Máximo
0,Precisão Falsas,1.0000,1.0000
1,Precisão Reais,0.7959,0.8161
2,Sensibilidade Reais,1.0000,1.0000
3,Sensibilidade Falsas,0.7730,0.7936
4,Acuracidade,0.8796,0.8920
